## Code for computing Gromov's delta on simulated data


In [143]:
# Torch device management
import torch

if torch.cuda.is_available():
    torch.cuda.set_device(0)
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")

Using device: cuda


##### We use the embedders package (https://github.com/pchlenski/embedders/) to simulate data

In [154]:
import networkx as nx
import random
import numpy as np
import embedders
import geoopt
import sys, os
sys.path.append('/home/raiyan/embedders/src/embedders')
from manifolds import Manifold, ProductManifold
from delta import hypdelta

In [149]:
N_ITERATIONS = 100

def sample_points_on_manifold(curvature, dim, n_points):
    """
    Samples points on a manifold based on the given curvatures.

    Args:
    - curvature: manifold curvature
    - dim: The dimension of the manifold.
    - n_points: The number of points to sample on each manifold.

    Returns:
    - the sampled points.
    """

    m_h = Manifold(K, dim)
    if K != 0:
        points = m_h.sample([([1] + [0] * dim) * n_points])
    else:
        points = m_h.sample([([1] + [0] * (dim - 1)) * n_points]) 

    distance_mat = m_h.dist(points[:, None], points[None, :])#m_h.pdist(points)
    distance_mat = distance_mat.detach().numpy()
    return points, distance_mat

### Compute delta-hyperbolicity of Spherical Spaces

In [153]:
dim = 2
n_points = 500
curvatures = [1, 2, 3]
deltas = []

for _ in range(N_ITERATIONS):

    for K in curvatures:
        points, distance_mat = sample_points_on_manifold(curvature=K, dim=dim, n_points=n_points)
    
        delta = hypdelta(distance_mat, device="cpu", strategy="CCL")
        deltas.append(delta)

print(f"The average delta hyperbolicity of randomly sampled points on spherical manifolds is: {np.mean(deltas)}")

The average delta hyperbolicity of randomly sampled points on spherical manifolds is: 0.9896971008577237


### Compute delta-hyperbolicity of Euclidean Space

In [140]:
from scipy.spatial import distance_matrix
dim = 2
n_points = 300
K=0

points, distance_mat = sample_points_on_manifold(curvature=K, dim=dim, n_points=n_points)

delta = hypdelta(distance_mat, device="cpu", strategy="naive")

print(f"The computed delta hyperbolicity of randomly sampled points at curvature {K} is: {delta}")

The computed delta hyperbolicity of randomly sampled points at curvature 0 is: 0.5185156364344387


In [61]:
from scipy.spatial import distance_matrix
dim = 2
n_points = 300
K=0

deltas = []

for _ in range(N_ITERATIONS):
    points = sample_points_on_manifold(curvature=K, dim=dim, n_points=n_points)
    
    points = points.detach().numpy()
    distance_mat = distance_matrix(points, points)#m_h.dist(points[:, None], points[None, :])
    #distance_mat = distance_mat.detach().numpy()
    delta = hypdelta(distance_mat, device="cpu", strategy="CCL")
    deltas.append(delta)

print(f"The computed delta hyperbolicity of randomly sampled points at curvature {K} is: {np.mean(deltas)}")

The computed delta hyperbolicity of randomly sampled points at curvature 0 is: 0.5125611688650199


In [138]:
from scipy.spatial import distance_matrix

dim = 2
n_points = 300
K=0
points, distance_mat1 = sample_points_on_manifold(curvature=K, dim=dim, n_points=n_points)


points = points.detach().numpy()
distance_mat2 = distance_matrix(points, points)

delta1 = hypdelta(distance_mat1, device="cpu", strategy="CCL")
delta2 = hypdelta(distance_mat2, device="cpu", strategy="CCL")

print(delta1, delta2)

0.5471001347339421 0.5471001340765046


### Compute delta-hyperbolicity of discrete spaces [tree]



In [155]:
def get_distance_matrix(G):
    """
    Calculate the distance matrix for a given graph G.

    Parameters:
    G (networkx.Graph): The input graph.

    Returns:
    numpy.ndarray: A distance matrix where the element at [i, j] is the shortest path distance between nodes i and j.
    """
    # Get the number of nodes
    num_nodes = G.number_of_nodes()
    
    # Initialize the distance matrix with infinity
    distance_matrix = np.full((num_nodes, num_nodes), np.inf)
    
    # Calculate shortest paths
    shortest_paths = dict(nx.all_pairs_shortest_path_length(G))
    
    # Fill the distance matrix
    for i in range(num_nodes):
        for j in range(num_nodes):
            if i in shortest_paths and j in shortest_paths[i]:
                distance_matrix[i, j] = shortest_paths[i][j]
            elif i == j:
                distance_matrix[i, j] = 0
    
    return distance_matrix


N_NODES = 100


deltas = []
for i in range(N_ITERATIONS):
    tree_graph = nx.random_tree(n=N_NODES)
    distance_mat = get_distance_matrix(tree_graph)
    delta = hypdelta(distance_mat, device="cpu", strategy="CCL")
    deltas.append(delta)

print(f"The computed delta hyperbolicity of random trees is: {np.mean(deltas)}")

The computed delta hyperbolicity of random trees is: 0.0


### Compute delta-hyperbolicity of discrete spaces [dense graph]

We use the following definition of a dense graph:

A graph $G = (V, E)$ is dense if for every $v \in V$, degree(v)$ > |V| / 2$

In [97]:
def create_random_regular_graph(num_vertices):
    if num_vertices % 2 != 0:
        raise ValueError("Number of vertices must be even for a regular graph with degree = num_vertices / 2 + 1")
    
    degree = (num_vertices // 2) + 1
    
    # Create a random regular graph
    G = nx.random_regular_graph(degree, num_vertices)
    
    # Shuffle the edges to make it more random
    edges = list(G.edges())
    random.shuffle(edges)
    G = nx.Graph()
    G.add_edges_from(edges)
    
    return G

In [118]:
# Example usage
num_vertices = 30
try:
    graph = create_random_regular_graph(num_vertices)
    print(f"Created a graph with {num_vertices} vertices")
    #print(f"Edges: {graph.edges()}")
    print(f"Degrees: {dict(graph.degree())}")
except ValueError as e:
    print(f"Error: {e}")

Created a graph with 100 vertices
Degrees: {26: 16, 27: 16, 21: 16, 10: 16, 24: 16, 9: 16, 7: 16, 23: 16, 5: 16, 25: 16, 19: 16, 22: 16, 20: 16, 12: 16, 28: 16, 13: 16, 2: 16, 6: 16, 8: 16, 4: 16, 14: 16, 18: 16, 1: 16, 3: 16, 0: 16, 29: 16, 16: 16, 17: 16, 11: 16, 15: 16}


In [121]:
num_vertices = 30
deltas = []
for i in range(N_ITERATIONS):
    graph = create_random_regular_graph(num_vertices)
    distance_mat = get_distance_matrix(graph)
    delta = hypdelta(distance_mat, device="cpu", strategy="CCL")
    deltas.append(delta)
print(np.mean(deltas))

0.8325
